<a href="https://colab.research.google.com/github/Artty02/Assignment/blob/main/Jang_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from datetime import date, timedelta

def get_days_after_startdate(month, year, startdate, days_of_week):
    """
    ฟังก์ชันนี้จะคืนรายการวันทั้งหมดในเดือนที่ระบุในปีที่ระบุ
    หลังจากวันที่เริ่มต้น (startdate) และแสดงเฉพาะวันที่อยู่ในรายการเลขที่กำหนด

    Args:
      month (int): เดือน (1-12)
      year (int): ปี
      startdate (date): วันที่เริ่มต้น (ไม่นับวันนี้)
      days_of_week (list): รายการเลขวันในสัปดาห์ (1-7)

    Returns:
      list: รายการวันหลังจาก startdate ที่ตรงกับ days_of_week
    """

    first_day_of_month = date(year, month, 1)
    last_day_of_month = (first_day_of_month + timedelta(days=32)).replace(day=1) - timedelta(days=1)

    days_in_month = []
    current_day = first_day_of_month
    while current_day <= last_day_of_month:
        if current_day > startdate:  # ปรับเงื่อนไขเพื่อไม่นับวันที่เริ่มต้น
            day_of_week = current_day.isoweekday()  # คืนค่าเลขวันในสัปดาห์ตามที่กำหนด
            if day_of_week in days_of_week:
                days_in_month.append(current_day)  # บันทึกแค่วันที่
        current_day += timedelta(days=1)

    return days_in_month

# รับ input เดือน และ ปี จากผู้ใช้
month = int(input("กรุณาป้อนเดือน (1-12): "))
year = int(input("กรุณาป้อนปี: "))

# อ่านไฟล์ Excel
df = pd.read_excel('Test working day.xlsx')

# สร้าง column ใหม่ 'add', 'delete', และ 'Genarate working day'
df['add'] = ''
df['delete'] = ''
df['Genarate working day'] = ''

# วนลูปในแต่ละแถวของ DataFrame
for index, row in df.iterrows():
    # ดึงวันที่เริ่มต้นจากคอลัมน์ A และแปลงเป็นวันที่
    startdate = pd.to_datetime(row['路由计划生效日期 eff.date']).date()

    # ดึง days_of_week จากคอลัมน์ C (ควรเป็น string ที่มีตัวเลขเช่น "135" -> [1, 3, 5])
    days_of_week_str = str(row['班期working day'])
    days_of_week = [int(x) for x in days_of_week_str if x.isdigit()]  # แยกเป็นตัวเลขแต่ละตัว

    # เรียกใช้ฟังก์ชัน get_days_after_startdate เพื่อสร้างวันที่ที่ทำงานในเดือนและปีที่ระบุ
    days = get_days_after_startdate(month, year, startdate, days_of_week)

    # จัดรูปแบบผลลัพธ์และบันทึกลงในคอลัมน์ 'Genarate working day'
    generated_days_str = ', '.join([day.strftime('%Y-%m-%d') for day in days])  # แสดงแค่วันที่
    df.loc[index, 'Genarate working day'] = generated_days_str

    # แยกตัวเลขใน column 'C' และ 'D' เพื่อคำนวณ 'add' และ 'delete'
    c_digits = list(str(row['班期working day']))
    d_digits = list(str(row['Old working day']))

    # หาตัวเลขที่อยู่ใน 'C' แต่ไม่อยู่ใน 'D' (add)
    added_digits = sorted(list(set(c_digits) - set(d_digits)))
    added_digits_str = ''.join(added_digits)
    df.loc[index, 'add'] = added_digits_str

    # หาตัวเลขที่อยู่ใน 'D' แต่ไม่อยู่ใน 'C' (delete)
    deleted_digits = sorted(list(set(d_digits) - set(c_digits)))
    deleted_digits_str = ''.join(deleted_digits)
    df.loc[index, 'delete'] = deleted_digits_str

# บันทึก DataFrame ที่อัปเดตลงในไฟล์ Excel ใหม่
df.to_excel('ติเองมั่ว.xlsx', index=False)


กรุณาป้อนเดือน (1-12): 9
กรุณาป้อนปี: 2024
